In [1]:
import FinanceDataReader as fdr
import pandas as pd
from datetime import datetime
import pandas_market_calendars as mcal
import warnings
import ta

from pykrx import stock
from defs import filter_df
from PublicDataReader import Fred
from marcap import marcap_data
from functools import reduce

In [2]:
def add_52_week_high_info(series):
    df = pd.DataFrame(series, columns=['Code'])
    print(df.head(4))
    high_52_week_list = []
    change_percentage_list = []

    for stock_code in series:
        # FinanceDataReader를 사용하여 주식 데이터 가져오기
        stock_data = fdr.DataReader(stock_code)
        
        # 52주 최고가 계산
        fifty_two_week_high = stock_data['Close'].rolling(window=252, min_periods=1).max()
        recent_52_week_high = fifty_two_week_high.iloc[-1]
        
        # 최근 종가 구하기
        recent_close = stock_data['Close'].iloc[-1]
        
        # 최근 종가 대비 52주 최고가 대비 변동율 계산
        change_percentage = ((recent_close - recent_52_week_high) / recent_52_week_high) * 100
        
        high_52_week_list.append(recent_52_week_high)
        change_percentage_list.append(change_percentage)

    df['52주 최고가'] = high_52_week_list
    df['52주 최고가 대비 변동율'] = change_percentage_list

    return df

kospi = fdr.StockListing('KOSPI')
kospi = kospi['Code']
kosdaq = fdr.StockListing('KOSDAQ')
kosdaq = kosdaq['Code']
code = pd.concat([kospi,kosdaq],axis=0)

# 데이터프레임 생성
result_df = add_52_week_high_info(code)

# 변동율을 기준으로 내림차순 정렬
sorted_df = result_df.sort_values(by='52주 최고가 대비 변동율', ascending=False)

# 상위 100개만 선택하여 새로운 데이터프레임 생성
top_200_df = sorted_df.head(200)
top_200_df = top_200_df.drop(['52주 최고가','52주 최고가 대비 변동율'],axis=1)

# 결과 출력
print(top_200_df)

     Code
0  005930
1  000660
2  373220
3  207940
        Code
97    005385
88    267260
486   090350
483   109070
1662  388800
...      ...
313   017940
467   005720
354   016800
1687  450940
1684  446150

[200 rows x 1 columns]
